In [6]:
import requests

In [7]:
import pandas as pd

# 인터넷에서 데이터 가져오기

In [8]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [11]:
header = {'User-Agent': 
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
req = requests.get(url, headers = header)
html = req.text


In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# 찾으려고 하는 데이터의 태그 찾기

In [14]:
# <table class="list-wrap">
table = soup.select_one('table.list-wrap')
trs = table.select('tr.list')
len(trs)

50

In [15]:
trs = soup.select('tr.list')
len(trs)

50

# 여러개의 데이터 중 하나를 선택해서 원하는 정보 추출

In [17]:
# Rank, Title, Artist, Album
tr = trs[0]
tr.select_one('.number').get_text()


'1\n                                        \n                                    \n유지\n\n'

In [18]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [19]:
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [20]:
rank = int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [21]:
# title
tr.select_one('.info').find('a').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                Queendom'

In [22]:
title = tr.select_one('.info').find('a').get_text().strip()
title

'Queendom'

In [24]:
# artist
tr.select_one('.info').select_one('.artist').get_text()

'Red Velvet (레드벨벳)'

In [25]:
# album
tr.select_one('.info').select_one('.albumtitle').get_text()

'Queendom - The 6th Mini Album'

In [28]:
album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
album

'Queendom - The 6th Mini Album'

## 한 페이지에 있는 모든 데이터를 반복문으로 가져오기

In [31]:
rank_list, title_list, artist_list, album_list = [],[], [],[]
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').find('a').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [32]:
df = pd.DataFrame({
      '순위':rank_list, '곡명':title_list,
      '가수':artist_list, '앨범':album_list
})
df.tail()

,순위,곡명,가수,앨범
45,46,사이렌 Remix (Feat. UNEDUCATED KID & Paul Blanco),호미들,사이렌 Remix
46,47,어푸 (Ah puh),아이유 (IU),IU 5th Album 'LILAC'
47,48,가을 우체국 앞에서,김대명,슬기로운 의사생활 시즌2 OST Part 2 (tvN 목요드라마)
48,49,Life Goes On,방탄소년단,BE
49,50,그대라는 사치,임영웅,사랑의 콜센타 PART62


In [34]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').find('a').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    lines.append([rank, title, artist, album])


In [36]:
df = pd.DataFrame(lines, columns=['순위','곡명','가수','앨범'])
df.head()

,순위,곡명,가수,앨범
0,1,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
1,2,신호등,이무진,신호등
2,3,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
3,4,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집
4,5,Weekend,태연 (TAEYEON),Weekend


# 모든 페이지 데이터 가져오기

In [37]:
sub = 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20210817&hh=14&rtm=Y&pg='
rank_list, title_list, artist_list, album_list = [],[], [],[]

for page in range(1,5):
    url = f'{sub}{page}'
    req = requests.get(url, headers = header)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
        title = tr.select_one('.info').find('a').get_text().strip()
        artist = tr.select_one('.info').select_one('.artist').get_text().strip()
        album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)

In [38]:
df = pd.DataFrame({
      '순위':rank_list, '곡명':title_list,
      '가수':artist_list, '앨범':album_list
})
df.tail()

,순위,곡명,가수,앨범
195,196,Downtown Baby,블루 (BLOO),Downtown Baby
196,197,노래방에서,장범준,장범준 3집
197,198,널 너무 모르고,헤이즈 (Heize),/// (너 먹구름 비)
198,199,"너에게 난, 나에게 넌",미도와 파라솔,슬기로운 의사생활 OST Part 12 (tvN 목요드라마)
199,200,Rose,디오 (D.O.),공감 - The 1st Mini Album


In [39]:
df.to_csv('GenieTop200(210817).csv', index=False)